# Session 1: Introduction to Gaussian Processes and PyMC

**Duration:** 3 hours  
**Workshop:** Gaussian Processes with PyMC and LLMs

---

Gaussian Processes (GPs) represent one of the most elegant and powerful tools in modern machine learning and statistics. Unlike parametric models that assume specific functional forms, GPs provide a **non-parametric approach** that can capture complex patterns while naturally quantifying uncertainty. This makes them particularly valuable for applications where understanding uncertainty is as important as making predictions—from scientific modeling to decision-making under uncertainty.

This session introduces the foundational concepts of Gaussian Processes within the PyMC probabilistic programming framework. We will build intuition about what it means for a process to be "Gaussian," explore the mathematical machinery that makes GPs work, and learn to implement them using PyMC's powerful and expressive interface.

## Why Gaussian Processes?

Traditional machine learning often focuses on finding the "best" parameters for a pre-specified model. Gaussian Processes take a fundamentally different approach: instead of assuming a specific functional form, they place a probability distribution directly over the **space of functions**. This perspective offers several compelling advantages:

- **Principled uncertainty quantification**: GPs provide natural confidence intervals and probability distributions over predictions
- **Automatic model selection**: Through marginal likelihood optimization, GPs can automatically tune their complexity to the data
- **Incorporation of prior knowledge**: Domain expertise can be encoded through choice of mean functions and covariance kernels
- **Small data efficiency**: GPs can make meaningful predictions and quantify uncertainty even with limited training data
- **Interpretable hyperparameters**: Kernel parameters often have clear physical or domain-specific meanings

## Learning Objectives

By the end of this session, you will be able to:

1. **Understand the mathematical foundations of GPs**: Grasp how Gaussian Processes extend multivariate Gaussian distributions to infinite-dimensional function spaces
2. **Build intuition through visualization**: Create and interpret samples from GP priors to understand how hyperparameters affect function behavior
3. **Master PyMC's probabilistic programming paradigm**: Use PyMC's model contexts, distributions, and inference machinery for GP modeling
4. **Construct and analyze covariance functions**: Build kernels from first principles and understand their role in encoding assumptions about function smoothness and structure
5. **Navigate PyMC's GP implementations**: Understand the trade-offs between `gp.Marginal` and `gp.Latent` approaches and when to use each
6. **Apply GPs to real problems**: Build complete GP regression models, from prior specification through posterior inference to prediction

---

## Environment Setup

We begin by setting up our computational environment with the necessary libraries for Gaussian Process modeling, Bayesian inference, and visualization. This section establishes the foundation for all subsequent analysis.

In [1]:
# Core scientific computing
import numpy as np
import scipy.stats as stats
import polars as pl

# PyMC ecosystem for probabilistic programming
import pymc as pm
import pytensor.tensor as pt
import arviz as az

# Visualization libraries
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

# Configure visualization defaults
az.style.use('arviz-doc')
pio.templates.default = 'plotly_white'
px.defaults.template = 'plotly_white'
px.defaults.width = 800
px.defaults.height = 500

# Reproducibility
RANDOM_SEED = 20090425
RNG = np.random.default_rng(RANDOM_SEED)

print(f"Environment configured successfully!")
print(f"PyMC version: {pm.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Random seed: {RANDOM_SEED}")

Environment configured successfully!
PyMC version: 5.25.1
NumPy version: 2.3.3
Random seed: 20090425


---

## Part I: Mathematical Foundations of Gaussian Processes

To understand Gaussian Processes deeply, we must first establish their mathematical foundations. This section will build intuition by connecting familiar concepts (univariate and multivariate Gaussians) to the more abstract notion of distributions over functions.

### From Scalars to Functions: The Gaussian Hierarchy

The conceptual progression from simple to complex Gaussian structures provides the key to understanding GPs:

1. **Univariate Gaussian**: $X \sim \mathcal{N}(\mu, \sigma^2)$ describes uncertainty about a single scalar value
2. **Multivariate Gaussian**: $\mathbf{X} \sim \mathcal{N}(\boldsymbol{\mu}, \Sigma)$ describes uncertainty about a finite-dimensional vector
3. **Gaussian Process**: $f(\cdot) \sim \mathcal{GP}(m(\cdot), k(\cdot, \cdot))$ describes uncertainty about an infinite-dimensional function

The remarkable insight of Gaussian Processes is that we can work with infinite-dimensional function spaces by considering only finite-dimensional marginals at any collection of input points.

### Formal Definition

**Definition**: A Gaussian Process is a collection of random variables, any finite number of which have a joint Gaussian distribution.

More precisely, a stochastic process $\{f(x) : x \in \mathcal{X}\}$ is a Gaussian Process if for any finite set of indices $\{x_1, x_2, \ldots, x_n\} \subset \mathcal{X}$, the joint distribution of the random vector $(f(x_1), f(x_2), \ldots, f(x_n))^T$ is multivariate Gaussian.

A Gaussian Process is completely specified by two functions:

1. **Mean function**: $m(x) = \mathbb{E}[f(x)]$
2. **Covariance function**: $k(x, x') = \mathbb{Cov}[f(x), f(x')] = \mathbb{E}[(f(x) - m(x))(f(x') - m(x'))]$

We denote this as:
$$f(x) \sim \mathcal{GP}(m(x), k(x, x'))$$

### The Finite-Dimensional View

For any finite collection of input points $\mathbf{X} = \{x_1, x_2, \ldots, x_n\}$, the corresponding function values $\mathbf{f} = [f(x_1), f(x_2), \ldots, f(x_n)]^T$ follow a multivariate Gaussian distribution:

$$\mathbf{f} \sim \mathcal{N}(\boldsymbol{\mu}, \mathbf{K})$$

where:
- $\boldsymbol{\mu} = [m(x_1), m(x_2), \ldots, m(x_n)]^T$ is the mean vector
- $\mathbf{K}$ is the covariance matrix with entries $K_{ij} = k(x_i, x_j)$

This finite-dimensional perspective is crucial because it allows us to:
- Sample functions from the GP (by sampling from the multivariate Gaussian)
- Compute likelihoods (using the multivariate Gaussian density)
- Perform inference (using standard multivariate Gaussian conditioning)

### Properties of Covariance Functions

The covariance function $k(x, x')$ is the heart of a Gaussian Process. It encodes our assumptions about function smoothness, periodicity, and other structural properties. For $k$ to be a valid covariance function, it must be:

1. **Symmetric**: $k(x, x') = k(x', x)$ for all $x, x'$
2. **Positive semi-definite**: For any finite set $\{x_1, \ldots, x_n\}$, the matrix $\mathbf{K}$ with $K_{ij} = k(x_i, x_j)$ must be positive semi-definite

These conditions ensure that the resulting covariance matrices are valid, guaranteeing that we can sample from and compute probabilities under the GP.

### Building Intuition: From Multivariate Gaussian to GP

Let's build intuition by starting with a simple multivariate Gaussian and then extending to the GP setting. We'll see how increasing the number of dimensions naturally leads us to the function space perspective.

In [2]:
def create_rbf_covariance(X, length_scale=1.0, variance=1.0):
    """
    Create RBF (Radial Basis Function) covariance matrix.
    
    The RBF kernel is defined as:
    k(x, x') = σ² * exp(-||x - x'||² / (2ℓ²))
    
    Parameters:
    -----------
    X : array-like, shape (n,)
        Input locations
    length_scale : float
        Length scale parameter ℓ
    variance : float  
        Variance parameter σ²
        
    Returns:
    --------
    K : ndarray, shape (n, n)
        Covariance matrix
    """
    X = np.asarray(X).reshape(-1, 1) if np.asarray(X).ndim == 1 else np.asarray(X)
    
    # Compute squared Euclidean distances
    sqdist = np.sum(X**2, axis=1)[:, None] + np.sum(X**2, axis=1)[None, :] - 2 * np.dot(X, X.T)
    
    # RBF covariance
    K = variance * np.exp(-0.5 * sqdist / length_scale**2)
    
    return K

def zero_mean_function(X):
    """Zero mean function."""
    return np.zeros(len(X))

# Demonstrate the progression from few to many points
fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=["5 points", "10 points", "25 points", "50 points"],
                    vertical_spacing=0.1)

n_points_list = [5, 10, 25, 50]
colors = ['blue', 'red', 'green', 'orange']

for idx, (n_points, color) in enumerate(zip(n_points_list, colors)):
    # Create input points
    X = np.linspace(-3, 3, n_points)
    
    # Create covariance matrix
    K = create_rbf_covariance(X, length_scale=1.0, variance=1.0)
    
    # Add small jitter for numerical stability
    K += 1e-6 * np.eye(len(X))
    
    # Sample functions
    mu = zero_mean_function(X)
    f_samples = RNG.multivariate_normal(mu, K, size=3)
    
    # Plot settings
    row = idx // 2 + 1
    col = idx % 2 + 1
    
    # Plot samples
    for i, f in enumerate(f_samples):
        fig.add_trace(
            go.Scatter(x=X, y=f, mode='lines+markers', 
                      line=dict(color=color, width=2),
                      marker=dict(size=4),
                      name=f"Sample {i+1}" if idx == 0 else None,
                      showlegend=idx == 0,
                      opacity=0.7),
            row=row, col=col
        )
    
    # Plot mean and confidence bands
    std = np.sqrt(np.diag(K))
    fig.add_trace(
        go.Scatter(x=np.concatenate([X, X[::-1]]),
                  y=np.concatenate([mu + 2*std, (mu - 2*std)[::-1]]),
                  fill='toself', fillcolor='rgba(128,128,128,0.2)',
                  line=dict(color='rgba(255,255,255,0)'),
                  name="±2σ" if idx == 0 else None,
                  showlegend=idx == 0),
        row=row, col=col
    )
    
    fig.add_trace(
        go.Scatter(x=X, y=mu, mode='lines',
                  line=dict(color='black', width=2, dash='dash'),
                  name="Mean" if idx == 0 else None,
                  showlegend=idx == 0),
        row=row, col=col
    )

fig.update_layout(
    height=600,
    title_text="Progression from Multivariate Gaussian to Gaussian Process",
    showlegend=True
)

fig.update_xaxes(title_text="Input x")
fig.update_yaxes(title_text="Function value f(x)")

fig.show()

print("As we increase the number of points, we approach a continuous function sampled from a GP.")
print("Each subplot shows 3 different function samples from the same GP prior.")

As we increase the number of points, we approach a continuous function sampled from a GP.
Each subplot shows 3 different function samples from the same GP prior.


**Key Insight**: As we increase the number of evaluation points, the discrete samples begin to resemble continuous functions. In the limit, we have a Gaussian Process that defines a probability distribution over the entire function space.

---

## Understanding GP Hyperparameters Through Function Behavior

The covariance function is completely characterized by its **hyperparameters**, and understanding how these parameters affect function behavior is crucial for effective GP modeling. The RBF (Radial Basis Function) kernel, also called the Exponentiated Quadratic or Squared Exponential kernel, has two key hyperparameters:

$$k(x, x') = \sigma^2 \exp\left(-\frac{||x - x'||^2}{2\ell^2}\right)$$

- **Length scale ($\ell$)**: Controls how quickly the correlation between function values decays with distance. Think of it as the "characteristic wiggle width"—smaller values produce rapidly varying functions, while larger values produce smoother functions.

- **Variance ($\sigma^2$)**: Controls the overall amplitude or vertical scale of functions. Larger variance allows functions to reach further from the mean.

The best way to understand these effects is not through abstract covariance matrices, but by looking at actual functions sampled from GP priors with different hyperparameter values. Let's visualize how each parameter shapes the kinds of functions the GP considers plausible.

In [ ]:
# Visualize the effect of LENGTH SCALE on GP prior samples
# We'll draw actual functions to build intuition about smoothness

X_plot = np.linspace(0, 10, 200)[:, None]

# Configuration: exploring length scale effect (keeping variance constant)
length_scales = [0.5, 1.0, 2.0, 4.0]
variance = 1.0

fig = make_subplots(rows=2, cols=2,
                    subplot_titles=[f"Length Scale ℓ = {ls}" for ls in length_scales],
                    vertical_spacing=0.12, horizontal_spacing=0.08)

for idx, ls in enumerate(length_scales):
    row = idx // 2 + 1
    col = idx % 2 + 1

    # Build covariance matrix
    K = create_rbf_covariance(X_plot.flatten(), length_scale=ls, variance=variance)
    K += 1e-6 * np.eye(len(X_plot))  # Numerical stability

    # Draw multiple samples from the GP prior
    mu = zero_mean_function(X_plot.flatten())
    samples = RNG.multivariate_normal(mu, K, size=5)

    # Plot each sample
    colors_samples = px.colors.qualitative.Set2
    for i, sample in enumerate(samples):
        fig.add_trace(
            go.Scatter(x=X_plot.flatten(), y=sample,
                      mode='lines',
                      line=dict(color=colors_samples[i % len(colors_samples)], width=1.5),
                      opacity=0.7,
                      showlegend=False),
            row=row, col=col
        )

    # Add zero mean line
    fig.add_trace(
        go.Scatter(x=X_plot.flatten(), y=mu,
                  mode='lines',
                  line=dict(color='black', width=1, dash='dash'),
                  showlegend=False),
        row=row, col=col
    )

fig.update_layout(
    height=700,
    title_text="Effect of Length Scale on GP Prior Samples (σ² = 1.0 fixed)",
    showlegend=False
)

fig.update_xaxes(title_text="x")
fig.update_yaxes(title_text="f(x)")

fig.show()

print("Length Scale Interpretation:")
print(f"• ℓ = {length_scales[0]}: Very short length scale → rapidly varying, wiggly functions")
print(f"• ℓ = {length_scales[1]}: Short length scale → moderately varying functions")
print(f"• ℓ = {length_scales[2]}: Medium length scale → smooth, gradually changing functions")
print(f"• ℓ = {length_scales[3]}: Long length scale → very smooth, slowly varying functions")

### Interpreting Length Scale Effects

Notice how dramatically the character of the sampled functions changes as we adjust the length scale parameter. This single hyperparameter essentially controls the "smoothness" or "wiggliness" of the functions:

- **Short length scales** (ℓ = 0.5, 1.0) produce functions that can change rapidly over small distances. The GP believes that nearby points might have quite different function values, leading to oscillatory behavior.

- **Long length scales** (ℓ = 2.0, 4.0) produce smooth, gradually varying functions. The GP believes that nearby points should have similar values, and it takes a substantial distance before the correlation decays significantly.

This parameter is often one of the most important to tune in GP applications. If your length scale is too short, the GP may overfit to noise; if it's too long, it may underfit and miss important variations in the data.

### Effect of Variance Parameter

Now let's examine how the variance (or amplitude) parameter $\sigma^2$ affects GP priors. Unlike length scale, which controls smoothness, variance controls the vertical scale—how far functions typically deviate from the mean.

In [ ]:
# Visualize the effect of VARIANCE on GP prior samples
# Variance controls the amplitude/vertical scale

# Configuration: exploring variance effect (keeping length scale constant)
variances = [0.25, 0.5, 1.0, 2.0]
length_scale = 1.5

fig = make_subplots(rows=2, cols=2,
                    subplot_titles=[f"Variance σ² = {var}" for var in variances],
                    vertical_spacing=0.12, horizontal_spacing=0.08)

for idx, var in enumerate(variances):
    row = idx // 2 + 1
    col = idx % 2 + 1

    # Build covariance matrix
    K = create_rbf_covariance(X_plot.flatten(), length_scale=length_scale, variance=var)
    K += 1e-6 * np.eye(len(X_plot))  # Numerical stability

    # Draw multiple samples from the GP prior
    mu = zero_mean_function(X_plot.flatten())
    samples = RNG.multivariate_normal(mu, K, size=5)

    # Plot each sample
    colors_samples = px.colors.qualitative.Plotly
    for i, sample in enumerate(samples):
        fig.add_trace(
            go.Scatter(x=X_plot.flatten(), y=sample,
                      mode='lines',
                      line=dict(color=colors_samples[i % len(colors_samples)], width=1.5),
                      opacity=0.7,
                      showlegend=False),
            row=row, col=col
        )

    # Add zero mean line
    fig.add_trace(
        go.Scatter(x=X_plot.flatten(), y=mu,
                  mode='lines',
                  line=dict(color='black', width=1, dash='dash'),
                  showlegend=False),
        row=row, col=col
    )

fig.update_layout(
    height=700,
    title_text="Effect of Variance on GP Prior Samples (ℓ = 1.5 fixed)",
    showlegend=False
)

fig.update_xaxes(title_text="x")
fig.update_yaxes(title_text="f(x)")

fig.show()

print("Variance Interpretation:")
print(f"• σ² = {variances[0]}: Low variance → functions stay close to the mean (y≈0)")
print(f"• σ² = {variances[1]}: Moderate variance → functions explore modest deviations")
print(f"• σ² = {variances[2]}: Standard variance → functions show typical variation")
print(f"• σ² = {variances[3]}: High variance → functions can deviate far from the mean")

### Understanding the Interplay of Hyperparameters

Looking at the variance effects, we see that this parameter acts as a "volume knob" for the GP. Unlike length scale, which controls the texture and smoothness of functions, variance scales their overall magnitude. Notice that the *character* of the functions remains consistent across variance levels—wiggly functions stay wiggly, smooth functions stay smooth—but their amplitude changes dramatically.

**Key Intuition**: Think of length scale as controlling the "horizontal resolution" (how quickly the function can change as you move along the x-axis) and variance as controlling the "vertical resolution" (how far the function can stray from the mean). Together, these two hyperparameters give you powerful control over the kinds of functions your GP prior considers plausible.

In practice, these hyperparameters are rarely set manually. Instead, we learn them from data through Bayesian inference, allowing the GP to automatically discover the appropriate smoothness and scale for the problem at hand. This is one of the key strengths of the Gaussian Process framework—it can adapt its complexity to match the data.

---

---

## 🤖 Hands-On Exercise 1: Using LLMs for Kernel Construction

Before we dive into PyMC specifics, let's practice using **Large Language Models (LLMs) like those in VSCode Copilot or Cursor** to help us understand and implement Gaussian Process kernels. This exercise demonstrates how to effectively collaborate with AI coding assistants for probabilistic programming.

### Exercise Instructions

**Your task**: Use your LLM to help you build and experiment with different kernel functions. Work through the following steps by asking your AI assistant for guidance:

1. **Ask your LLM**: "Help me implement a custom RBF kernel function in Python that matches PyMC's ExpQuad kernel"
2. **Request comparison**: "Show me how to compare my custom kernel with PyMC's built-in kernel"
3. **Seek visualization help**: "Create a function to visualize how different hyperparameters affect kernel shape"

### Effective Prompting Tips for PyMC/GP Problems

When working with LLMs on Gaussian Process problems, use these strategies:

- **Be specific about the framework**: Mention "PyMC", "Gaussian Processes", and specific function names
- **Include context**: "I'm working on a regression problem with X inputs and y outputs"
- **Request explanations**: Ask "Why did you choose this kernel?" or "What does this hyperparameter control?"
- **Ask for alternatives**: "What other kernels could work for this problem?"
- **Request debugging help**: "This PyMC model isn't converging, what should I check?"

### Sample Prompts to Try

Copy and paste these prompts into your LLM assistant (modify with your specific details):

```
PROMPT 1: "I'm learning Gaussian Processes with PyMC. Can you help me implement 
a function that creates an RBF covariance matrix from scratch, then compare it 
to PyMC's pm.gp.cov.ExpQuad? Include visualization of the kernel shape."

PROMPT 2: "Show me how to sample functions from a Gaussian Process prior 
using my custom kernel. I want to see how changing the lengthscale from 0.1 
to 2.0 affects the smoothness of sampled functions."

PROMPT 3: "Help me understand why my PyMC GP model is taking a long time 
to sample. I'm using 100 data points with an ExpQuad kernel. What are 
common performance issues and how can I optimize it?"
```

**Work on the exercise below, but don't hesitate to ask your LLM for help when you get stuck!**

In [4]:
# 🤖 EXERCISE: Use your LLM to help complete this kernel implementation

# STEP 1: Ask your LLM to help you implement this function
def custom_rbf_kernel(X1, X2, lengthscale=1.0, variance=1.0):
    """
    Custom RBF kernel implementation - ask your LLM to help complete this!
    
    Prompt suggestion: "Help me complete this RBF kernel function that computes
    the covariance matrix between input points X1 and X2 with given lengthscale
    and variance parameters. The formula is: k(x,x') = σ² * exp(-||x-x'||²/(2ℓ²))"
    """
    # YOUR LLM-ASSISTED CODE HERE
    pass

# STEP 2: Ask your LLM to help you create a comparison with PyMC's kernel
def compare_with_pymc_kernel(X_test):
    """
    Compare custom kernel with PyMC's ExpQuad - get LLM help here!
    
    Prompt suggestion: "Show me how to compare my custom RBF kernel with
    PyMC's pm.gp.cov.ExpQuad kernel using the same hyperparameters. Create
    a visualization that shows both kernel shapes side by side."
    """
    # YOUR LLM-ASSISTED CODE HERE
    pass

# STEP 3: Ask your LLM to create a hyperparameter sensitivity analysis
def analyze_hyperparameter_effects():
    """
    Analyze how lengthscale and variance affect kernel behavior.
    
    Prompt suggestion: "Create an interactive visualization showing how
    different lengthscale and variance values affect the RBF kernel shape
    and sampled functions. Include at least 3 different parameter combinations."
    """
    # YOUR LLM-ASSISTED CODE HERE
    pass

# Test your implementations
X_demo = np.linspace(-3, 3, 50)[:, None]

print("🎯 Exercise Goal: Use your LLM assistant to implement the functions above!")
print("💡 Remember to ask for explanations of any code you don't understand.")
print("🔍 Try different prompting strategies to see which work best for you.")

🎯 Exercise Goal: Use your LLM assistant to implement the functions above!
💡 Remember to ask for explanations of any code you don't understand.
🔍 Try different prompting strategies to see which work best for you.


---

## Part II: PyMC Fundamentals for Probabilistic Programming

Before diving into Gaussian Processes specifically, we need to understand PyMC's approach to probabilistic programming. PyMC provides a powerful framework for specifying, fitting, and analyzing Bayesian models through an intuitive Python interface.

### The Philosophy of Probabilistic Programming

Probabilistic programming represents a paradigm shift in statistical modeling. Instead of deriving update equations or coding samplers by hand, we declare the structure of our model and let the framework handle the computational details. This approach offers several advantages:

- **Model specification mirrors mathematical notation**: Code looks like the mathematical model
- **Automatic inference**: No need to implement custom sampling algorithms
- **Composability**: Complex models can be built from simpler components
- **Flexibility**: Easy to experiment with different model structures

### PyMC's Core Components

PyMC organizes probabilistic models around several key abstractions:

1. **Model Context**: A context manager that tracks all model components
2. **Random Variables**: Represent uncertain quantities with probability distributions
3. **Deterministic Variables**: Represent quantities that are functions of other variables
4. **Observed Variables**: Represent data that we condition on

Let's explore each of these concepts through examples.

### Model Contexts and Random Variables

Every PyMC model exists within a **Model context**. This context manager keeps track of all model components and their relationships:

In [5]:
# Create a simple model context
with pm.Model() as simple_model:
    # Define a random variable
    theta = pm.Normal('theta', mu=0, sigma=1)
    
    # The model automatically tracks this variable
    print(f"Model variables: {list(simple_model.named_vars.keys())}")
    print(f"Variable type: {type(theta)}")
    
# We can examine the model structure
print(f"\nModel summary:")
print(f"Number of free random variables: {len(simple_model.free_RVs)}")
print(f"Number of observed variables: {len(simple_model.observed_RVs)}")

Model variables: ['theta']
Variable type: <class 'pytensor.tensor.variable.TensorVariable'>

Model summary:
Number of free random variables: 1
Number of observed variables: 0


### Working with Distributions

PyMC provides a comprehensive library of probability distributions. Let's explore some commonly used distributions and their properties:

In [6]:
# Demonstrate different distribution types
with pm.Model() as distribution_demo:
    
    # Continuous distributions
    normal_var = pm.Normal('normal', mu=0, sigma=1)
    gamma_var = pm.Gamma('gamma', alpha=2, beta=1)
    beta_var = pm.Beta('beta', alpha=2, beta=2)
    
    # Discrete distributions  
    binomial_var = pm.Binomial('binomial', n=10, p=0.3)
    poisson_var = pm.Poisson('poisson', mu=3)
    
    # Half-distributions (positive support)
    half_normal_var = pm.HalfNormal('half_normal', sigma=1)
    
    print("Distribution types in the model:")
    for var_name, var in distribution_demo.named_vars.items():
        print(f"  {var_name}: {var.owner.op.__class__.__name__}")

Distribution types in the model:
  normal: NormalRV
  gamma: GammaRV
  beta: BetaRV
  binomial: BinomialRV
  poisson: PoissonRV
  half_normal: HalfNormalRV


### Sampling from Distributions

PyMC provides several ways to sample from distributions. The `pm.draw()` function allows us to sample from the prior distributions:

In [7]:
# Sample from distributions
with distribution_demo:
    # Sample single values
    print("Single samples:")
    print(f"Normal: {pm.draw(normal_var):.3f}")
    print(f"Gamma: {pm.draw(gamma_var):.3f}")
    print(f"Beta: {pm.draw(beta_var):.3f}")
    print(f"Binomial: {pm.draw(binomial_var)}")
    print(f"Poisson: {pm.draw(poisson_var)}")
    
    # Sample multiple values
    normal_samples = pm.draw(normal_var, draws=1000)
    print(f"\n1000 Normal samples - Mean: {normal_samples.mean():.3f}, Std: {normal_samples.std():.3f}")

Single samples:
Normal: -0.942
Gamma: 1.820
Beta: 0.459
Binomial: 3
Poisson: 4

1000 Normal samples - Mean: 0.021, Std: 1.024


### Computing Log-Probabilities

A fundamental operation in Bayesian inference is computing log-probabilities. PyMC provides the `pm.logp()` function for this purpose:

In [8]:
# Compute log-probabilities
with distribution_demo:
    # Evaluate log-probability at specific values
    print("Log-probabilities:")
    print(f"Normal(0) at x=0: {pm.logp(normal_var, 0).eval():.3f}")
    print(f"Normal(0) at x=2: {pm.logp(normal_var, 2).eval():.3f}")
    print(f"Gamma(α=2,β=1) at x=1: {pm.logp(gamma_var, 1).eval():.3f}")
    print(f"Beta(α=2,β=2) at x=0.5: {pm.logp(beta_var, 0.5).eval():.3f}")
    
    # Compare to scipy for verification
    scipy_normal_logpdf = stats.norm.logpdf(0, loc=0, scale=1)
    pymc_normal_logp = pm.logp(normal_var, 0).eval()
    print(f"\nVerification - SciPy: {scipy_normal_logpdf:.6f}, PyMC: {pymc_normal_logp:.6f}")

Log-probabilities:
Normal(0) at x=0: -0.919
Normal(0) at x=2: -2.919
Gamma(α=2,β=1) at x=1: -1.000
Beta(α=2,β=2) at x=0.5: 0.405

Verification - SciPy: -0.918939, PyMC: -0.918939


### Deterministic Variables and Transformations

Often we need to create variables that are deterministic functions of other variables. PyMC provides two approaches: anonymous transformations and named deterministic variables:

In [9]:
with pm.Model() as transformation_model:
    # Base random variables
    x = pm.Normal('x', mu=0, sigma=1)
    y = pm.Normal('y', mu=0, sigma=1)
    
    # Anonymous transformation (not tracked in output)
    z_anonymous = x + y  # This won't appear in sampling output
    
    # Named deterministic (tracked in output)
    z_named = pm.Deterministic('sum_xy', x + y)
    squared = pm.Deterministic('x_squared', x**2)
    
    # We can also create more complex transformations
    complex_transform = pm.Deterministic('complex', 
                                       pt.sin(x) * pt.exp(y / 2))

print("Variables in transformation model:")
for name in transformation_model.named_vars.keys():
    print(f"  {name}")

Variables in transformation model:
  x
  y
  sum_xy
  x_squared
  complex


### Prior Predictive Sampling

Before fitting models to data, it's crucial to understand what our priors imply. **Prior predictive sampling** generates data from our model before seeing any observations:

In [ ]:
# Create a simple linear regression model for demonstration
with pm.Model() as linear_model:
    # Priors for regression coefficients
    alpha = pm.Normal('intercept', mu=0, sigma=1)
    beta = pm.Normal('slope', mu=0, sigma=1)
    sigma = pm.HalfNormal('sigma', sigma=1)
    
    # Create some input data
    x_data = np.linspace(-2, 2, 50)
    
    # Define the linear relationship
    mu = pm.Deterministic('mu', alpha + beta * x_data)
    
    # Likelihood (but no observed data yet)
    y = pm.Normal('y', mu=mu, sigma=sigma)
    
    # Sample from the prior predictive distribution
    prior_predictive = pm.sample_prior_predictive(samples=500, random_seed=RANDOM_SEED)

# Visualize prior predictive samples
# In PyMC 5.25+, prior predictive samples are stored in the 'prior' group
y_samples = prior_predictive.prior['y'].values

fig = go.Figure()

# Plot several prior predictive realizations
for i in range(min(20, y_samples.shape[0])):
    fig.add_trace(
        go.Scatter(x=x_data, y=y_samples[i, 0, :],
                  mode='lines', opacity=0.3,
                  line=dict(color='blue'),
                  showlegend=i==0, name='Prior samples')
    )

fig.update_layout(
    title="Prior Predictive Samples from Linear Regression Model",
    xaxis_title="x",
    yaxis_title="y",
    height=500
)

fig.show()

print(f"Prior predictive samples shape: {y_samples.shape}")
print(f"This shows {y_samples.shape[0]} different realizations of our prior beliefs")

### Parameter Constraints and Transformations

Many parameters have natural constraints (e.g., variances must be positive). PyMC automatically handles these constraints through parameter transformations:

In [ ]:
with pm.Model() as constrained_model:
    # Constrained variables
    positive_var = pm.HalfNormal('positive', sigma=1)  # x >= 0
    bounded_var = pm.Beta('bounded', alpha=2, beta=2)  # 0 <= x <= 1
    unrestricted_var = pm.Normal('unrestricted', mu=0, sigma=1)  # x ∈ ℝ
    
    # PyMC automatically creates transformed versions for sampling
    print("Free (transformed) variables for sampling:")
    for rv in constrained_model.free_RVs:
        print(f"  {rv}")
    
    print("\nValue variables (original scale):")
    for rv in constrained_model.value_vars:
        print(f"  {rv}")

**Key Point**: PyMC handles parameter transformations automatically. For example, `HalfNormal` variables are log-transformed during sampling to ensure they remain positive, then back-transformed for interpretation.

---

## Part III: Introduction to PyMC Gaussian Processes

Now that we understand PyMC's fundamentals, let's explore how to work with Gaussian Processes. PyMC provides a comprehensive GP module (`pm.gp`) with implementations optimized for different use cases.

### PyMC's GP Module Structure

PyMC's GP functionality is organized into several key components:

1. **Mean Functions** (`pm.gp.mean`): Define the expected function behavior
2. **Covariance Functions** (`pm.gp.cov`): Define the correlation structure
3. **GP Implementations**: Different computational approaches
   - `pm.gp.Marginal`: Efficient for Gaussian likelihoods
   - `pm.gp.Latent`: Flexible for non-Gaussian likelihoods

### Mean Functions

Mean functions specify the expected value of the GP at each input. Let's explore the built-in options:

In [ ]:
# Create demonstration data
X_demo = np.linspace(0, 10, 100)[:, None]

# Different mean functions
mean_functions = {
    'Zero': pm.gp.mean.Zero(),
    'Constant': pm.gp.mean.Constant(c=2.5),
    'Linear': pm.gp.mean.Linear(coeffs=pt.as_tensor([0.5]), intercept=pt.as_tensor(1.0))
}

# Evaluate mean functions
fig = go.Figure()

colors = ['blue', 'red', 'green']
for (name, mean_func), color in zip(mean_functions.items(), colors):
    mean_values = mean_func(X_demo).eval()
    
    fig.add_trace(
        go.Scatter(x=X_demo.flatten(), y=mean_values,
                  mode='lines', name=f'{name} Mean',
                  line=dict(color=color, width=3))
    )

fig.update_layout(
    title="PyMC Mean Functions",
    xaxis_title="x",
    yaxis_title="Mean function value m(x)",
    height=400
)

fig.show()

print("Mean Functions in PyMC:")
print("• Zero(): m(x) = 0 for all x")
print("• Constant(c): m(x) = c for all x")
print("• Linear(coeffs, intercept): m(x) = intercept + coeffs·x")
print("• And more: Polynomial, custom functions...")

### Covariance Functions (Kernels)

Covariance functions are the heart of GP modeling. They encode our assumptions about function behavior. Let's explore PyMC's built-in kernels:

In [ ]:
# Demonstrate different covariance functions
x_test = np.array([[0.0]])  # Reference point
X_range = np.linspace(-3, 3, 200)[:, None]

# Different covariance functions with similar length scales
kernels = {
    'ExpQuad (RBF)': pm.gp.cov.ExpQuad(1, ls=1.0),
    'Matérn 5/2': pm.gp.cov.Matern52(1, ls=1.0),
    'Matérn 3/2': pm.gp.cov.Matern32(1, ls=1.0),
    'Exponential': pm.gp.cov.Exponential(1, ls=1.0)
}

fig = go.Figure()

colors = ['blue', 'red', 'green', 'orange']
for (name, kernel), color in zip(kernels.items(), colors):
    # Compute covariance with reference point
    cov_values = [kernel(x_test, x).eval().item() for x in X_range]
    
    fig.add_trace(
        go.Scatter(x=X_range.flatten(), y=cov_values,
                  mode='lines', name=name,
                  line=dict(color=color, width=3))
    )

fig.update_layout(
    title="Covariance Functions: k(0, x) vs x",
    xaxis_title="Distance from reference point",
    yaxis_title="Covariance k(0, x)",
    height=500
)

fig.show()

print("Kernel Properties:")
print("• ExpQuad: Infinitely differentiable (very smooth functions)")
print("• Matérn 5/2: Twice differentiable (smooth functions)")
print("• Matérn 3/2: Once differentiable (moderately smooth)")
print("• Exponential: Continuous but not differentiable (rough functions)")

---

## 🤖 Hands-On Exercise 2: Building PyMC GP Models with LLM Assistance

Now let's practice using LLMs to help us build complete PyMC Gaussian Process models. This exercise will guide you through using AI assistants to construct, debug, and optimize GP models.

### Exercise Objectives

1. Use your LLM to help implement both Marginal and Latent GP approaches
2. Practice debugging PyMC model issues with AI assistance
3. Learn to ask effective questions about hyperparameter selection
4. Get help interpreting convergence diagnostics

### Advanced Prompting Strategies for PyMC GPs

When asking your LLM for help with PyMC GP models, try these specific approaches:

**For Model Building:**
- "I have [X-type] data with [Y observations]. Help me choose between pm.gp.Marginal and pm.gp.Latent"
- "Show me how to set up a PyMC model with a [kernel-type] covariance function for [problem-type]"

**For Debugging:**
- "My PyMC GP model has [specific error]. Here's my code: [paste code]. What's wrong?"
- "PyMC sampling is very slow with [N] data points. How can I optimize this GP model?"

**For Analysis:**
- "Help me interpret these PyMC convergence diagnostics: [paste results]"
- "My GP predictions look wrong. How should I validate this PyMC model?"

### Specific Prompts for This Exercise

Try these prompts with your LLM (adapt as needed):

```
PROMPT 1: "Help me create a PyMC Gaussian Process regression model using
pm.gp.Marginal with an ExpQuad kernel. I want to model some 1D noisy sine wave data.
Include proper hyperparameter priors and show how to sample from the posterior."

PROMPT 2: "Now help me implement the same model using pm.gp.Latent instead.
Explain the differences in computational cost and when I'd choose each approach.
Include posterior predictive sampling."

PROMPT 3: "My PyMC GP model gives R-hat values above 1.1. Help me diagnose
and fix convergence issues. What should I check and how can I improve sampling?"

PROMPT 4: "Show me how to compare two different kernel choices (RBF vs Matérn)
for the same dataset using PyMC. Include model comparison metrics like WAIC."
```

In [ ]:
# 🤖 EXERCISE: Use your LLM to help build complete PyMC GP models

# Generate some synthetic data for the exercise
np.random.seed(42)
X_train = np.linspace(0, 2*np.pi, 15)[:, None]
y_train = np.sin(X_train.flatten()) + 0.2 * RNG.standard_normal(15)
X_test = np.linspace(-0.5, 2.5*np.pi, 100)[:, None]

print("Dataset created: 15 noisy sine wave observations")
print("Your task: Use your LLM to help you model this data with GPs!")

# TASK 1: Ask your LLM to help implement a Marginal GP model
def build_marginal_gp_model():
    """
    Use your LLM to help create a PyMC Marginal GP regression model.
    
    Suggested prompt: "Help me create a PyMC model using pm.gp.Marginal to fit
    a noisy sine wave dataset. Use an ExpQuad kernel with appropriate priors.
    Show me how to fit the model and generate predictions."
    """
    # YOUR LLM-ASSISTED CODE HERE
    pass

# TASK 2: Ask your LLM to help implement a Latent GP model
def build_latent_gp_model():
    """
    Use your LLM to help create a PyMC Latent GP regression model.
    
    Suggested prompt: "Now help me implement the same regression problem
    using pm.gp.Latent instead of pm.gp.Marginal. Explain when I should
    use each approach and show the computational differences."
    """
    # YOUR LLM-ASSISTED CODE HERE
    pass

# TASK 3: Ask your LLM for help with model diagnostics
def diagnose_model_convergence(trace):
    """
    Get LLM help to check model convergence and sampling quality.
    
    Suggested prompt: "Help me create a function that checks PyMC sampling
    convergence for GP models. Include R-hat, ESS, and visual diagnostics.
    Show me what values indicate good vs. poor convergence."
    """
    # YOUR LLM-ASSISTED CODE HERE
    pass

# TASK 4: Get LLM help for kernel comparison
def compare_kernel_choices():
    """
    Use your LLM to help compare different kernel choices.
    
    Suggested prompt: "Help me compare ExpQuad vs. Matern52 kernels
    on the same dataset using PyMC. Show me how to compute model
    comparison metrics and visualize the differences in predictions."
    """
    # YOUR LLM-ASSISTED CODE HERE
    pass

print("🎯 Goal: Complete all 4 tasks using your LLM assistant!")
print("🤔 Don't just copy code - ask your LLM to explain each step.")
print("🔍 Experiment with different prompting approaches to see what works best.")

### Expected Learning Outcomes

After completing this LLM-assisted exercise, you should be able to:

- **Effectively prompt** LLMs for PyMC-specific help
- **Debug common issues** in GP models with AI assistance
- **Compare different implementations** (Marginal vs. Latent) intelligently
- **Interpret model diagnostics** with LLM guidance
- **Iterate on model designs** using AI feedback

Remember: The goal isn't to have the LLM write all your code, but to use it as a knowledgeable pair programmer that can help you understand concepts, debug issues, and explore alternatives!

---

## Part IV: GP Implementations in PyMC - A Real Example

Now let's put everything together by building a complete GP regression model using real data. We'll demonstrate both the Marginal and Latent approaches and compare their performance.

### Creating Synthetic Regression Data

Let's create a realistic regression dataset that will showcase the strengths of GP modeling:

In [ ]:
# Generate synthetic regression data with non-linear structure
def true_function(x):
    """A complex non-linear function to learn."""
    return (0.8 * np.sin(2*np.pi*x) + 
            0.3 * np.cos(6*np.pi*x) + 
            0.1 * x**2 - 0.05 * x)

# Training data - deliberately sparse to show GP uncertainty
n_train = 20
X_train = RNG.uniform(0, 1, n_train)[:, None]
X_train = np.sort(X_train, axis=0)

y_true = true_function(X_train.flatten())
noise_std = 0.08
y_train = y_true + RNG.normal(0, noise_std, n_train)

# Test data for predictions
X_test = np.linspace(-0.1, 1.1, 150)[:, None]  # Slightly outside training range
y_test_true = true_function(X_test.flatten())

# Visualize the data
fig = go.Figure()

# True function
fig.add_trace(
    go.Scatter(x=X_test.flatten(), y=y_test_true,
              mode='lines', name='True function',
              line=dict(color='black', width=3, dash='dash'))
)

# Training data
fig.add_trace(
    go.Scatter(x=X_train.flatten(), y=y_train,
              mode='markers', name='Training data',
              marker=dict(color='red', size=10, symbol='circle'))
)

# True (noiseless) training points
fig.add_trace(
    go.Scatter(x=X_train.flatten(), y=y_true,
              mode='markers', name='True (noiseless)',
              marker=dict(color='darkred', size=8, symbol='x'))
)

fig.update_layout(
    title="Synthetic Regression Dataset",
    xaxis_title="x",
    yaxis_title="y",
    height=500,
    legend=dict(x=0.02, y=0.98)
)

fig.show()

print(f"Training data: {n_train} points")
print(f"Noise standard deviation: {noise_std}")
print(f"Training range: [{X_train.min():.2f}, {X_train.max():.2f}]")
print(f"Test range: [{X_test.min():.2f}, {X_test.max():.2f}] (includes extrapolation)")

### Approach 1: Marginal GP

The marginal approach analytically integrates out the latent function, making it computationally efficient for Gaussian likelihoods:

In [ ]:
with pm.Model() as marginal_model:
    
    # Hyperpriors for kernel hyperparameters
    # Length scale: how quickly the covariance decays
    ℓ = pm.InverseGamma("ℓ", alpha=5, beta=5)  # Weakly informative
    
    # Marginal standard deviation: overall function scale
    η = pm.HalfNormal("η", sigma=2)
    
    # Observation noise standard deviation
    σ = pm.HalfNormal("σ", sigma=0.5)
    
    # Mean function (zero for simplicity)
    mean_func = pm.gp.mean.Zero()
    
    # Covariance function: scaled Matérn 5/2 kernel
    cov_func = η**2 * pm.gp.cov.Matern52(1, ℓ)
    
    # GP prior specification
    gp = pm.gp.Marginal(mean_func=mean_func, cov_func=cov_func)
    
    # Marginal likelihood - integrates out the function analytically
    y_obs = gp.marginal_likelihood("y", X=X_train, y=y_train, sigma=σ)
    
    print("Marginal GP Model Structure:")
    print(f"Hyperparameters: {[v.name for v in marginal_model.free_RVs]}")
    print(f"Total free parameters: {len(marginal_model.free_RVs)}")
    print("Note: Latent function values are integrated out analytically")

### Approach 2: Latent GP

The latent approach explicitly includes the function values as parameters, providing more flexibility but at higher computational cost:

In [ ]:
with pm.Model() as latent_model:
    
    # Same hyperpriors
    ℓ = pm.InverseGamma("ℓ", alpha=5, beta=5)
    η = pm.HalfNormal("η", sigma=2)
    σ = pm.HalfNormal("σ", sigma=0.5)
    
    # Same mean and covariance functions
    mean_func = pm.gp.mean.Zero()
    cov_func = η**2 * pm.gp.cov.Matern52(1, ℓ)
    
    # GP specification
    gp = pm.gp.Latent(mean_func=mean_func, cov_func=cov_func)
    
    # Explicit prior over function values at training points
    f = gp.prior("f", X=X_train)
    
    # Likelihood connecting function values to observations
    y_obs = pm.Normal("y", mu=f, sigma=σ, observed=y_train)
    
    print("Latent GP Model Structure:")
    print(f"Hyperparameters: {[v.name for v in latent_model.free_RVs if v.name != 'f']}")
    print(f"Function values: f (dimension {f.eval().shape})")
    print(f"Total free parameters: {len(latent_model.free_RVs)}")
    print("Note: Function values are explicit random variables")

### Model Fitting and Performance Comparison

Let's fit both models and compare their computational performance:

In [ ]:
# Fit the marginal model
print("Fitting Marginal GP model...")
import time
start_time = time.time()

with marginal_model:
    trace_marginal = pm.sample(
        draws=1000,
        tune=1000,
        chains=2,
        target_accept=0.95,
        random_seed=RANDOM_SEED,
        progressbar=False
    )

marginal_time = time.time() - start_time
marginal_ess = az.ess(trace_marginal).min().values

print(f"✓ Marginal model fitted in {marginal_time:.1f}s")
print(f"  Minimum ESS: {marginal_ess:.0f}")
print(f"  ESS per second: {marginal_ess/marginal_time:.1f}")

In [ ]:
# Fit the latent model
print("\nFitting Latent GP model...")
start_time = time.time()

with latent_model:
    trace_latent = pm.sample(
        draws=1000,
        tune=1000,
        chains=2,
        target_accept=0.95,
        random_seed=RANDOM_SEED,
        progressbar=False
    )

latent_time = time.time() - start_time
latent_ess = az.ess(trace_latent, var_names=['ℓ', 'η', 'σ']).min().values

print(f"✓ Latent model fitted in {latent_time:.1f}s")
print(f"  Minimum ESS (hyperparameters): {latent_ess:.0f}")
print(f"  ESS per second: {latent_ess/latent_time:.1f}")
print(f"\nSpeedup factor: {latent_time/marginal_time:.1f}x (Marginal is faster)")

### Generating Predictions

Now let's generate predictions from both models and compare their performance:

In [ ]:
# Generate predictions from both models
print("Generating predictions...")

# Marginal model predictions
with marginal_model:
    f_pred_marginal = gp.conditional("f_pred", X_test)
    pred_marginal = pm.sample_posterior_predictive(
        trace_marginal,
        var_names=["f_pred"],
        progressbar=False,
        random_seed=RANDOM_SEED
    )

# Latent model predictions  
with latent_model:
    f_pred_latent = gp.conditional("f_pred", X_test)
    pred_latent = pm.sample_posterior_predictive(
        trace_latent,
        var_names=["f_pred"],
        progressbar=False,
        random_seed=RANDOM_SEED
    )

# Extract prediction statistics
f_pred_marginal_samples = pred_marginal.posterior_predictive["f_pred"].values
f_pred_mean_marginal = f_pred_marginal_samples.mean(axis=(0, 1))
f_pred_std_marginal = f_pred_marginal_samples.std(axis=(0, 1))

f_pred_latent_samples = pred_latent.posterior_predictive["f_pred"].values
f_pred_mean_latent = f_pred_latent_samples.mean(axis=(0, 1))
f_pred_std_latent = f_pred_latent_samples.std(axis=(0, 1))

print("✓ Predictions generated for both models")

### Results Visualization and Comparison

Let's create a comprehensive comparison of both approaches:

In [ ]:
# Create comprehensive comparison plot
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=["Marginal GP Predictions", "Latent GP Predictions",
                                   "Residuals Comparison", "Uncertainty Comparison"],
                    vertical_spacing=0.1, horizontal_spacing=0.1)

# Function to add GP predictions to subplot
def add_gp_predictions(fig, row, col, X, y_true, y_pred_mean, y_pred_std, 
                      X_train, y_train, color, name_prefix):
    # Confidence interval
    fig.add_trace(
        go.Scatter(
            x=np.concatenate([X.flatten(), X.flatten()[::-1]]),
            y=np.concatenate([y_pred_mean + 2*y_pred_std,
                             (y_pred_mean - 2*y_pred_std)[::-1]]),
            fill='toself',
            fillcolor=f'rgba({"0,100,255" if color=="blue" else "0,200,100"},0.3)',
            line=dict(color='rgba(255,255,255,0)'),
            showlegend=False,
            hoverinfo='skip'
        ),
        row=row, col=col
    )
    
    # Prediction mean
    fig.add_trace(
        go.Scatter(x=X.flatten(), y=y_pred_mean,
                  mode='lines', name=f'{name_prefix} Mean',
                  line=dict(color=color, width=2),
                  showlegend=row==1 and col==1),
        row=row, col=col
    )
    
    # True function
    fig.add_trace(
        go.Scatter(x=X.flatten(), y=y_true,
                  mode='lines', name='True Function',
                  line=dict(color='black', width=2, dash='dash'),
                  showlegend=row==1 and col==1),
        row=row, col=col
    )
    
    # Training data
    fig.add_trace(
        go.Scatter(x=X_train.flatten(), y=y_train,
                  mode='markers', name='Training Data',
                  marker=dict(color='red', size=6),
                  showlegend=row==1 and col==1),
        row=row, col=col
    )

# Add predictions for both models
add_gp_predictions(fig, 1, 1, X_test, y_test_true, f_pred_mean_marginal, 
                  f_pred_std_marginal, X_train, y_train, 'blue', 'Marginal')
add_gp_predictions(fig, 1, 2, X_test, y_test_true, f_pred_mean_latent,
                  f_pred_std_latent, X_train, y_train, 'green', 'Latent')

# Residuals comparison
residuals_marginal = f_pred_mean_marginal - y_test_true
residuals_latent = f_pred_mean_latent - y_test_true

fig.add_trace(
    go.Scatter(x=X_test.flatten(), y=residuals_marginal,
              mode='lines', name='Marginal Residuals',
              line=dict(color='blue', width=2)),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=X_test.flatten(), y=residuals_latent,
              mode='lines', name='Latent Residuals',
              line=dict(color='green', width=2)),
    row=2, col=1
)
fig.add_hline(y=0, line=dict(color='black', dash='dash'), row=2, col=1)

# Uncertainty comparison
fig.add_trace(
    go.Scatter(x=X_test.flatten(), y=f_pred_std_marginal,
              mode='lines', name='Marginal Std',
              line=dict(color='blue', width=2)),
    row=2, col=2
)
fig.add_trace(
    go.Scatter(x=X_test.flatten(), y=f_pred_std_latent,
              mode='lines', name='Latent Std', 
              line=dict(color='green', width=2)),
    row=2, col=2
)

# Update layout
fig.update_layout(
    height=800,
    title_text="Comprehensive GP Model Comparison",
    showlegend=True
)

fig.update_xaxes(title_text="x")
fig.update_yaxes(title_text="y", row=1)
fig.update_yaxes(title_text="Residual", row=2, col=1)
fig.update_yaxes(title_text="Standard Deviation", row=2, col=2)

fig.show()

# Performance metrics
mse_marginal = np.mean(residuals_marginal**2)
mse_latent = np.mean(residuals_latent**2)
mae_marginal = np.mean(np.abs(residuals_marginal))
mae_latent = np.mean(np.abs(residuals_latent))

print("\n" + "="*50)
print("PERFORMANCE COMPARISON")
print("="*50)
print(f"Marginal GP:")
print(f"  MSE: {mse_marginal:.6f}")
print(f"  MAE: {mae_marginal:.6f}")
print(f"  Sampling time: {marginal_time:.1f}s")
print(f"\nLatent GP:")
print(f"  MSE: {mse_latent:.6f}")
print(f"  MAE: {mae_latent:.6f}")
print(f"  Sampling time: {latent_time:.1f}s")
print(f"\nDifference:")
print(f"  ΔMSE: {abs(mse_marginal - mse_latent):.6f}")
print(f"  Speed ratio: {latent_time/marginal_time:.1f}x")

### Hyperparameter Posterior Analysis

Let's examine the learned hyperparameters from both models:

In [ ]:
# Compare hyperparameter posteriors
fig = make_subplots(rows=1, cols=3,
                    subplot_titles=["Length Scale (ℓ)", "Marginal Std (η)", "Noise Std (σ)"])

# Extract samples
marginal_samples = az.extract(trace_marginal, num_samples=1000)
latent_samples = az.extract(trace_latent, num_samples=1000, var_names=['ℓ', 'η', 'σ'])

params = ['ℓ', 'η', 'σ']
colors = ['blue', 'green']
names = ['Marginal', 'Latent']

for i, param in enumerate(params):
    # Marginal samples
    fig.add_trace(
        go.Histogram(
            x=marginal_samples[param].values,
            name=names[0] if i == 0 else None,
            opacity=0.7,
            nbinsx=30,
            marker_color=colors[0],
            showlegend=i==0
        ),
        row=1, col=i+1
    )
    
    # Latent samples
    fig.add_trace(
        go.Histogram(
            x=latent_samples[param].values,
            name=names[1] if i == 0 else None,
            opacity=0.7,
            nbinsx=30,
            marker_color=colors[1],
            showlegend=i==0
        ),
        row=1, col=i+1
    )

fig.update_layout(
    height=400,
    title_text="Hyperparameter Posterior Distributions",
    barmode='overlay'
)

fig.update_xaxes(title_text="Parameter value")
fig.update_yaxes(title_text="Frequency")

fig.show()

# Posterior summaries
print("\nHyperparameter Posterior Summaries:")
print("\nMarginal GP:")
for param in params:
    samples = marginal_samples[param].values
    mean_val = samples.mean()
    std_val = samples.std()
    q025, q975 = np.percentile(samples, [2.5, 97.5])
    print(f"  {param}: {mean_val:.3f} ± {std_val:.3f} [{q025:.3f}, {q975:.3f}]")

print("\nLatent GP:")
for param in params:
    samples = latent_samples[param].values
    mean_val = samples.mean()
    std_val = samples.std()
    q025, q975 = np.percentile(samples, [2.5, 97.5])
    print(f"  {param}: {mean_val:.3f} ± {std_val:.3f} [{q025:.3f}, {q975:.3f}]")

---

## Summary: When to Use Which Approach?

Based on our comprehensive comparison, here are the key decision criteria:

### Use **Marginal GP** (`pm.gp.Marginal`) when:
- ✅ **Gaussian likelihood**: You have regression with normal noise
- ✅ **Computational efficiency**: Speed and memory are important
- ✅ **Large datasets**: More than ~100-200 data points
- ✅ **Standard regression**: Basic function interpolation/extrapolation
- ✅ **Production deployment**: Need fast inference

### Use **Latent GP** (`pm.gp.Latent`) when:
- ✅ **Non-Gaussian likelihoods**: Classification, count data, robust regression
- ✅ **Function access needed**: Want posterior samples of the function itself
- ✅ **Complex models**: Hierarchical models, multi-output GPs
- ✅ **Small datasets**: Fewer than ~100 data points
- ✅ **Research/exploration**: Flexibility more important than speed

### Key Takeaways

1. **Performance**: Both approaches yield virtually identical predictions for Gaussian regression
2. **Speed**: Marginal approach is typically 2-5x faster
3. **Memory**: Marginal approach uses less memory (O(n²) vs O(n² + n))
4. **Flexibility**: Latent approach works with any likelihood
5. **Hyperparameters**: Both learn very similar hyperparameter values

### Best Practices

- **Start with Marginal GP** for standard regression problems
- **Use informative priors** on hyperparameters when possible
- **Check prior predictive samples** before fitting
- **Monitor convergence** using effective sample size and R-hat
- **Validate predictions** on held-out test data

---

## Next Steps and Advanced Topics

Congratulations! You've mastered the fundamentals of Gaussian Process modeling with PyMC. You now understand:

✅ **Mathematical foundations**: From multivariate Gaussians to function-space distributions  
✅ **PyMC fundamentals**: Model contexts, distributions, and probabilistic programming  
✅ **Kernel theory**: How covariance functions encode function properties  
✅ **Implementation trade-offs**: When to use marginal vs latent approaches  
✅ **Complete workflow**: From prior specification to posterior analysis  

### Preview of Session 2

In the next session, **"Advanced Kernels and Applications"**, we'll explore:

- **Kernel composition**: Combining kernels for complex patterns
- **Specialized kernels**: Periodic, polynomial, and custom kernels
- **Multi-dimensional inputs**: Handling higher-dimensional data
- **Non-Gaussian likelihoods**: Classification and count data
- **Model selection**: Comparing and validating GP models
- **Scalability**: Techniques for larger datasets

### Practice Exercises with LLM Assistance

Now that you understand the fundamentals, it's time to practice using **Large Language Models (LLMs) like those in VSCode/Cursor to assist with PyMC GP development**. These exercises are specifically designed to help you leverage AI coding assistants effectively for probabilistic programming.

1. **LLM-Assisted Kernel Experimentation**: Ask your LLM to help implement different kernels (Matérn, Periodic, combinations) and explain how they affect model behavior
2. **Interactive Hyperparameter Analysis**: Use your LLM to create visualizations showing how different prior choices affect GP predictions
3. **Real Data Application**: Have your LLM help you process and model a real dataset with appropriate GP specifications
4. **Model Comparison**: Ask your LLM to implement and compare different GP formulations on the same problem

### Resources

- **PyMC GP Documentation**: [Official Guide](https://www.pymc.io/projects/docs/en/stable/api/gp.html)
- **Textbook**: Rasmussen & Williams (2006) "Gaussian Processes for Machine Learning" 
- **Examples**: [PyMC GP Gallery](https://www.pymc.io/projects/examples/en/latest/gaussian_processes/index.html)

You've built a solid foundation for advanced GP modeling!